In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifierAE import TrashClassifier
from ai.DataLoader2 import DataLoader
from env import *

EPOCHS = 200
ETA = 1e-2

In [2]:
model = TrashClassifier(fine_tune=False)
model = model.cuda()

FeatureAE was loaded.


In [3]:
train_dloader = DataLoader(TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(VALID_DATA_PATH, TRASH_CAT)

Number of data: 540
Number of batch: 67
Number of data: 80
Number of batch: 10


In [4]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [5]:

top_valid_acc = 0.0

for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        with torch.no_grad():
            model.eval()

            for x_batch, y_batch in valid_dloader.next_batch():
                x_batch = torch.FloatTensor(x_batch).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                logps = model(x_batch)
                loss = criterion(logps, y_batch)

                valid_loss += loss.item()

                ps = torch.exp(logps)
                ps_k, cls_k = ps.topk(1, dim=1)
                equal = cls_k == y_batch.view(*cls_k.size())
                valid_acc += torch.mean(equal.type(torch.FloatTensor))

            train_loss /= len(train_dloader)
            train_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_acc:.8f}")
            
            if valid_acc > top_valid_acc:
                model.save(CLF_CKPT_PATH)
                top_valid_acc = valid_acc

            model.train()

Epochs: 10/200
Train loss: 0.25646217
Train acc: 0.91044772
Valid loss: 1.60298763
Valid acc: 0.73333341
Trash Classifier was saved.
Epochs: 20/200
Train loss: 0.18144579
Train acc: 0.92288542
Valid loss: 0.50677464
Valid acc: 0.86666667
Trash Classifier was saved.
Epochs: 30/200
Train loss: 0.13762293
Train acc: 0.95771140
Valid loss: 1.75968016
Valid acc: 0.71666670
Epochs: 40/200
Train loss: 0.11859368
Train acc: 0.96268654
Valid loss: 0.87479118
Valid acc: 0.83333337
Epochs: 50/200
Train loss: 0.15878998
Train acc: 0.95771128
Valid loss: 1.01331448
Valid acc: 0.78333336
Epochs: 60/200
Train loss: 0.14146035
Train acc: 0.95522374
Valid loss: 0.80555591
Valid acc: 0.86666662
Epochs: 70/200
Train loss: 0.22536439
Train acc: 0.93532330
Valid loss: 1.05947494
Valid acc: 0.75000006
Epochs: 80/200
Train loss: 0.11803909
Train acc: 0.96766162
Valid loss: 0.99078529
Valid acc: 0.80000001
Epochs: 90/200
Train loss: 0.04876082
Train acc: 0.98756218
Valid loss: 1.44889359
Valid acc: 0.78333336

KeyboardInterrupt: 

In [ ]:
1+2